In [3]:
!pip install gtts moviepy
!pip install flask pyngrok transformers torch nltk
!ngrok authtoken 2njsJ6qq6IcTrn9tQsLfD7KTmKx_5ztbtrkJFJgeECe9gfGrW
!pip install gtts moviepy
!pip install flask pyngrok transformers torch nltk
!ngrok authtoken 2njsJ6qq6IcTrn9tQsLfD7KTmKx_5ztbtrkJFJgeECe9gfGrW
!pip install pdfkit
!apt-get install -y wkhtmltopdf
!pip install beautifulsoup4 requests

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy libavahi-core7 libavahi-glib1
  libdaemon0 libevdev2 libfontenc1 libgudev-1.0-0 libhyphen0 libinput-bin libinput10
  libjson-glib-1.0-0 libjson-glib-1.0-common liblmdb0 libmaxminddb0 libmbim-glib4 libmbim-proxy
  libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200 libnotify4 libnss-mdns libproxy1v5 libqmi-glib5
  libqmi-proxy libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5
  libqt5printsupport5 libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1 libsoup2.

In [13]:
import requests
from gtts import gTTS
from moviepy.editor import *
import os
from flask import Flask, render_template_string, request, send_file
from bs4 import BeautifulSoup

# Unsplash API configuration
UNSPLASH_ACCESS_KEY = "CRhb-o5AP9jrv3h-QhBMslPHvoCd7chi2_OVRzVdHt0"

# Initialize Flask app
app = Flask(__name__)

# Helper function to fetch images from Unsplash based on keywords
def fetch_images_from_unsplash(keywords, num_images=3):
    images = []
    for keyword in keywords:
        response = requests.get(
            f"https://api.unsplash.com/search/photos",
            params={"query": keyword, "per_page": num_images},
            headers={"Authorization": f"Client-ID {UNSPLASH_ACCESS_KEY}"}
        )
        data = response.json()
        for photo in data['results']:
            images.append(photo['urls']['regular'])
        if len(images) >= num_images:
            break
    return images[:num_images]

# Helper function to generate video from text and images
def generate_video_from_text_and_images(summary, key_points):
    # Combine the summary and key points into a single string
    text_to_speech = f"Summary: {summary}. Key Points: {', '.join(key_points)}."

    # Generate speech using gTTS
    tts = gTTS(text=text_to_speech, lang='en')
    audio_file = "output_audio.mp3"
    tts.save(audio_file)

    # Fetch images from Unsplash based on key points
    keywords = summary.split()[:3]  # Use first 3 words of the summary as keywords
    images = fetch_images_from_unsplash(keywords)

    # Create clips for each image and set their duration based on the audio length
    audio_clip = AudioFileClip(audio_file)
    image_clips = []
    for img_url in images:
        image = ImageClip(img_url).set_duration(audio_clip.duration / len(images))
        image = image.resize(height=720)  # Resize images to fit the video
        image_clips.append(image)

    # Concatenate the image clips into a video
    video = concatenate_videoclips(image_clips, method="compose").set_audio(audio_clip)

    video_file = "study_material_video.mp4"
    video.write_videofile(video_file, fps=24)

    # Clean up the audio file after video generation
    os.remove(audio_file)

    return video_file

# Helper function to scrape content from GeeksforGeeks based on the topic
def scrape_content(topic):
    search_url = f"https://www.geeksforgeeks.org/?s={topic.replace(' ', '+')}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the first article link
    first_article_link = soup.find('a', href=True)
    article_url = first_article_link['href']

    article_response = requests.get(article_url)
    article_soup = BeautifulSoup(article_response.content, 'html.parser')

    # Extract paragraphs from the article
    paragraphs = article_soup.find_all('p')
    content = " ".join([para.get_text() for para in paragraphs])

    return content

# Dummy functions to simulate the summary, key points, and questions generation
def generate_summary(content):
    return "This is a dummy summary of the topic based on the content."

def generate_key_points(content):
    return ["Key Point 1", "Key Point 2", "Key Point 3"]

def generate_questions(content):
    return ["What is Key Point 1?", "How does Key Point 2 work?", "Explain Key Point 3."]

# Main function to generate study material
def generate_study_material(content):
    return {
        'Summary': generate_summary(content),
        'Key Points': generate_key_points(content),
        'Questions': generate_questions(content)
    }

# Route to display the form and show generated study material
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        topic = request.form['topic']

        # Scrape content based on the topic
        scraped_content = scrape_content(topic)

        # Generate study material from the scraped content
        study_material = generate_study_material(scraped_content)

        return render_template_string('''
            <div class="container mt-5">
                <h1 class="text-center mb-4">Study Material Generator</h1>
                <form method="POST" class="mb-5">
                    <div class="form-group">
                        <label for="topic">Enter the Topic:</label>
                        <input type="text" class="form-control" id="topic" name="topic" value="{{ topic }}">
                    </div>
                    <button type="submit" class="btn btn-primary btn-block">Generate Study Material</button>
                </form>

                <div class="card">
                    <div class="card-body">
                        <h4 class="card-title">Summary</h4>
                        <p class="card-text">{{ study_material['Summary'] }}</p>
                    </div>
                </div>

                <div class="card mt-4">
                    <div class="card-body">
                        <h4 class="card-title">Key Points</h4>
                        <ul>
                            {% for point in study_material['Key Points'] %}
                                <li>{{ point }}</li>
                            {% endfor %}
                        </ul>
                    </div>
                </div>

                <div class="card mt-4">
                    <div class="card-body">
                        <h4 class="card-title">Questions</h4>
                        <ul>
                            {% for question in study_material['Questions'] %}
                                <li>{{ question }}</li>
                            {% endfor %}
                        </ul>
                    </div>
                </div>

                <form method="POST" action="/generate_video">
                    <input type="hidden" name="summary" value="{{ study_material['Summary'] }}">
                    <input type="hidden" name="key_points" value="{{ study_material['Key Points']|join(',') }}">
                    <button type="submit" class="btn btn-success mt-4">Generate Video Lecture</button>
                </form>
            </div>
        ''', study_material=study_material, topic=topic)

    return render_template_string('''
        <div class="container mt-5">
            <h1 class="text-center mb-4">Study Material Generator</h1>
            <form method="POST" class="mb-5">
                <div class="form-group">
                    <label for="topic">Enter the Topic:</label>
                    <input type="text" class="form-control" id="topic" name="topic" placeholder="Enter a topic...">
                </div>
                <button type="submit" class="btn btn-primary btn-block">Generate Study Material</button>
            </form>
        </div>
    ''')

# Route to generate video lecture from the study material
@app.route('/generate_video', methods=['POST'])
def generate_video():
    summary = request.form['summary']
    key_points = request.form['key_points'].split(',')

    # Generate video from the summary, key points, and images
    video_file = generate_video_from_text_and_images(summary, key_points)

    # Send the generated video file as a download
    return send_file(video_file, as_attachment=True)

# Run Flask app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
